# Benchmarking study

**Corresponding to Section 5.2 in the accompanying report**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/samuelbarrett1234/atml-group-11/blob/main/damon/notebooks/experiments.ipynb)


## Colab admin

To run in Colab switch to a GPU runtime, execute the below two cells, then restart your runtime.

In [ ]:
import torch
TORCH = torch.__version__.split("+")[0]
CUDA = "cu"+torch.version.cuda.replace(".","")

%pip install torch-scatter \
             torch-sparse \
             torch-cluster \
             torch-spline-conv \
             -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-geometric 

In [ ]:
# %pip install --no-cache cloud-tpu-client==0.10 torch==1.10 https://storage.googleapis.com/tpu-pytorch/wheels/colab/1.10/torch_xla-1.10-cp37-cp37m-linux_x86_64.whl
!git clone https://github.com/samuelbarrett1234/atml-group-11.git
%pip install atml-group-11/damon/src

### To reload `oxgat`

If the repo has changed, do the following to update the package (make sure to restart the runtime):

In [ ]:
%cd atml-group-11
!git pull origin main
%cd ..
%pip uninstall oxgat
%pip install atml-group-11/damon/src

## Setup

In [1]:
import oxgat
import torch
from torch_geometric.datasets import Planetoid, Amazon, CoraFull, Coauthor
import torch_geometric.utils
from torch_geometric.transforms import NormalizeFeatures
from ogb.graphproppred import PygGraphPropPredDataset
import json

**At any point run the following cell to visualise logged training progress:**

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

## Load data

In [2]:
torch.manual_seed(0)

# Transductive node classification
cora = Planetoid("./data", name="Cora", transform=NormalizeFeatures())
citeseer = Planetoid("./data", name="CiteSeer", transform=NormalizeFeatures())
pubmed = Planetoid("./data", name="PubMed", transform=NormalizeFeatures())
corafull = CoraFull("./data", transform=NormalizeFeatures())
photo = Amazon("./data", name="Photo", transform=NormalizeFeatures())
computers = Amazon("./data", name="Computers", transform=NormalizeFeatures())
compsci = Coauthor("./data", name="CS", transform=NormalizeFeatures())
physics = Coauthor("./data", name="Physics", transform=NormalizeFeatures())

# # Graph classification
molhiv = PygGraphPropPredDataset(name="ogbg-molhiv")
split_idx = molhiv.get_idx_split()
molhiv_train = molhiv[split_idx["train"]]
molhiv_val = molhiv[split_idx["valid"]]
molhiv_test = molhiv[split_idx["test"]]

Add train-val-test splits to datasets that don't already have them:

In [ ]:
def add_masks(data):
    num_nodes = data.x.size(0)
    num_classes = data.y.unique().size(0)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    for c in range(num_classes):
        relevant_nodes = (data.y == c).nonzero().flatten()
        n = min(relevant_nodes.size(0), 20)
        to_include = relevant_nodes[torch.randperm(relevant_nodes.size(0))[:n]]
        mask = torch_geometric.utils.index_to_mask(to_include, size=num_nodes)
        train_mask = torch.logical_or(train_mask, mask)

    unused_nodes = torch.logical_not(train_mask).nonzero().flatten()
    val_nodes = unused_nodes[torch.randperm(unused_nodes.size(0))[:500]]
    val_mask = torch_geometric.utils.index_to_mask(val_nodes, size=num_nodes)

    unused_nodes = torch.logical_not(torch.logical_or(train_mask, val_mask)).nonzero().flatten()
    test_nodes = unused_nodes[torch.randperm(unused_nodes.size(0))[:1000]]
    test_mask = torch_geometric.utils.index_to_mask(val_nodes, size=num_nodes)

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    return data

corafull = [add_masks(corafull[0])]
photo = [add_masks(photo[0])]
computers = [add_masks(computers[0])]
compsci = [add_masks(compsci[0])]
physics = [add_masks(physics[0])]

## GAT

In [ ]:
gat_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gat_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 92.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 92.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 92.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8289999961853027}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gat_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  237 K │
└───┴────────┴────────────┴────────┘

Trainable params: 237 K                                                                      
Non-trainable params: 0                                                                      
Total params: 237 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7039999961853027}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gat_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 32.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 32.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 32.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.777999997138977}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gat_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  562 K │
└───┴────────┴────────────┴────────┘

Trainable params: 562 K                                                                      
Non-trainable params: 0                                                                      
Total params: 562 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.1899999976158142}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gat_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 48.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 48.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 48.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8659999966621399}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gat_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 49.9 K │
└───┴────────┴────────────┴────────┘

Trainable params: 49.9 K                                                                     
Non-trainable params: 0                                                                      
Total params: 49.9 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7519999742507935}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gat_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  436 K │
└───┴────────┴────────────┴────────┘

Trainable params: 436 K                                                                      
Non-trainable params: 0                                                                      
Total params: 436 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8999999761581421}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gat_results["physics"] = physics_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  539 K │
└───┴────────┴────────────┴────────┘

Trainable params: 539 K                                                                      
Non-trainable params: 0                                                                      
Total params: 539 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9240000247955322}

--------------------------------------------------------------------------------

In [16]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  restore_best="acc",
                                                  metric="roc_auc_score")
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gat_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  1.2 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 1.9 K                                                                      
Non-trainable params: 0                                                                      
Total params: 1.9 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.5632553738001892}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gat_results.json", "w") as f:
    json.dump(gat_results, f)

## GAT with 2-hop neighbourhoods

In [ ]:
gatd2_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               neighbourhood_depth=2,
                                               sparse=False)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatd2_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 92.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 92.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 92.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7929999828338623}

--------------------------------------------------------------------------------

In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   neighbourhood_depth=2,
                                                   sparse=False)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatd2_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  237 K │
└───┴────────┴────────────┴────────┘

Trainable params: 237 K                                                                      
Non-trainable params: 0                                                                      
Total params: 237 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7099999785423279}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 neighbourhood_depth=2,
                                                 sparse=False)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatd2_results["pubmed"] = pubmed_model.final_metrics

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   neighbourhood_depth=2,
                                                   sparse=False)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatd2_results["corafull"] = corafull_model.final_metrics

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                neighbourhood_depth=2,
                                                sparse=False)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatd2_results["photo"] = photo_model.final_metrics

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    neighbourhood_depth=2,
                                                    sparse=False)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatd2_results["computers"] = computers_model.final_metrics

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  neighbourhood_depth=2,
                                                  sparse=False)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatd2_results["compsci"] = compsci_model.final_metrics

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  neighbourhood_depth=2,
                                                  sparse=False)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatd2_results["physics"] = physics_model.final_metrics

In [ ]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  restore_best="acc",
                                                  metric="roc_auc_score",
                                                  neighbourhood_depth=2,
                                                  sparse=False)
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatd2_results["molhiv"] = molhiv_model.final_metrics

In [ ]:
with open("results/gatd2_results.json", "w") as f:
    json.dump(gatd2_results, f)

## GAT without self-loops

In [ ]:
gatnsl_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               strict_neighbourhoods=True)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatnsl_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 92.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 92.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 92.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8140000104904175}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   strict_neighbourhoods=True)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatnsl_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  237 K │
└───┴────────┴────────────┴────────┘

Trainable params: 237 K                                                                      
Non-trainable params: 0                                                                      
Total params: 237 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6790000200271606}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 strict_neighbourhoods=True)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatnsl_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 32.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 32.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 32.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7730000019073486}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   strict_neighbourhoods=True)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatnsl_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  562 K │
└───┴────────┴────────────┴────────┘

Trainable params: 562 K                                                                      
Non-trainable params: 0                                                                      
Total params: 562 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.20600000023841858}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                strict_neighbourhoods=True)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatnsl_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 48.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 48.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 48.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8500000238418579}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    strict_neighbourhoods=True)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatnsl_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 49.9 K │
└───┴────────┴────────────┴────────┘

Trainable params: 49.9 K                                                                     
Non-trainable params: 0                                                                      
Total params: 49.9 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7319999933242798}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  strict_neighbourhoods=True)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatnsl_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  436 K │
└───┴────────┴────────────┴────────┘

Trainable params: 436 K                                                                      
Non-trainable params: 0                                                                      
Total params: 436 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8880000114440918}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  strict_neighbourhoods=True)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatnsl_results["physics"] = physics_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  539 K │
└───┴────────┴────────────┴────────┘

Trainable params: 539 K                                                                      
Non-trainable params: 0                                                                      
Total params: 539 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9160000085830688}

--------------------------------------------------------------------------------

In [17]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  strict_neighbourhoods=True)
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatnsl_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  1.2 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 1.9 K                                                                      
Non-trainable params: 0                                                                      
Total params: 1.9 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6404159987639777}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatnsl_results.json", "w") as f:
    json.dump(gatnsl_results, f)

## GAT with degree encoding

In [ ]:
gatde_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                               max_degree=oxgat.utils.get_max_degree(cora))
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatde_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  104 K │
└───┴────────┴────────────┴────────┘

Trainable params: 104 K                                                                      
Non-trainable params: 0                                                                      
Total params: 104 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.765999972820282}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                   max_degree=oxgat.utils.get_max_degree(citeseer))
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatde_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  244 K │
└───┴────────┴────────────┴────────┘

Trainable params: 244 K                                                                      
Non-trainable params: 0                                                                      
Total params: 244 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6150000095367432}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                 max_degree=oxgat.utils.get_max_degree(pubmed))
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatde_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 43.9 K │
└───┴────────┴────────────┴────────┘

Trainable params: 43.9 K                                                                     
Non-trainable params: 0                                                                      
Total params: 43.9 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.640999972820282}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                   max_degree=oxgat.utils.get_max_degree(corafull))
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatde_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  602 K │
└───┴────────┴────────────┴────────┘

Trainable params: 602 K                                                                      
Non-trainable params: 0                                                                      
Total params: 602 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.335999995470047}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                max_degree=oxgat.utils.get_max_degree(photo))
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatde_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  151 K │
└───┴────────┴────────────┴────────┘

Trainable params: 151 K                                                                      
Non-trainable params: 0                                                                      
Total params: 151 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7699999809265137}

--------------------------------------------------------------------------------

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                    max_degree=oxgat.utils.get_max_degree(computers))
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatde_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  271 K │
└───┴────────┴────────────┴────────┘

Trainable params: 271 K                                                                      
Non-trainable params: 0                                                                      
Total params: 271 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7099999785423279}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                  max_degree=oxgat.utils.get_max_degree(compsci))
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatde_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  447 K │
└───┴────────┴────────────┴────────┘

Trainable params: 447 K                                                                      
Non-trainable params: 0                                                                      
Total params: 447 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8339999914169312}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                  max_degree=oxgat.utils.get_max_degree(physics))
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatde_results["physics"] = physics_model.final_metrics

In [13]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  layer_type=oxgat.components.MultiHeadAttentionLayerWithDegrees,
                                                  max_degree=max(oxgat.utils.get_max_degree(molhiv_train),
                                                                 oxgat.utils.get_max_degree(molhiv_val),
                                                                 oxgat.utils.get_max_degree(molhiv_test)))
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatde_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  2.0 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 2.7 K                                                                      
Non-trainable params: 0                                                                      
Total params: 2.7 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6921802275053593}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatde_results.json", "w") as f:
    json.dump(gatde_results, f)

## GATv2 normal (weight-sharing, bias)

In [ ]:
gatv2_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               attention_type=oxgat.components.GATv2AttentionHead)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatv2_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 92.4 K │
└───┴────────┴────────────┴────────┘

Trainable params: 92.4 K                                                                     
Non-trainable params: 0                                                                      
Total params: 92.4 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8399999737739563}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   attention_type=oxgat.components.GATv2AttentionHead)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatv2_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  237 K │
└───┴────────┴────────────┴────────┘

Trainable params: 237 K                                                                      
Non-trainable params: 0                                                                      
Total params: 237 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7049999833106995}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 attention_type=oxgat.components.GATv2AttentionHead)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatv2_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 32.4 K │
└───┴────────┴────────────┴────────┘

Trainable params: 32.4 K                                                                     
Non-trainable params: 0                                                                      
Total params: 32.4 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7829999923706055}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   attention_type=oxgat.components.GATv2AttentionHead)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatv2_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  562 K │
└───┴────────┴────────────┴────────┘

Trainable params: 562 K                                                                      
Non-trainable params: 0                                                                      
Total params: 562 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.28600001335144043}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                attention_type=oxgat.components.GATv2AttentionHead)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatv2_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 48.4 K │
└───┴────────┴────────────┴────────┘

Trainable params: 48.4 K                                                                     
Non-trainable params: 0                                                                      
Total params: 48.4 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8740000128746033}

--------------------------------------------------------------------------------

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    attention_type=oxgat.components.GATv2AttentionHead)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatv2_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 50.0 K │
└───┴────────┴────────────┴────────┘

Trainable params: 50.0 K                                                                     
Non-trainable params: 0                                                                      
Total params: 50.0 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.628000020980835}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  attention_type=oxgat.components.GATv2AttentionHead)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatv2_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  436 K │
└───┴────────┴────────────┴────────┘

Trainable params: 436 K                                                                      
Non-trainable params: 0                                                                      
Total params: 436 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9139999747276306}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  attention_type=oxgat.components.GATv2AttentionHead)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatv2_results["physics"] = physics_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  539 K │
└───┴────────┴────────────┴────────┘

Trainable params: 539 K                                                                      
Non-trainable params: 0                                                                      
Total params: 539 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9240000247955322}

--------------------------------------------------------------------------------

In [4]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  attention_type=oxgat.components.GATv2AttentionHead)
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatv2_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  1.3 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 2.0 K                                                                      
Non-trainable params: 0                                                                      
Total params: 2.0 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6770273663068039}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatv2_results.json", "w") as f:
    json.dump(gatv2_results, f)

## GATv2 (weight-sharing, no bias)

In [ ]:
gatv2nb_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               attention_type=oxgat.components.GATv2AttentionHead,
                                               bias=False)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatv2nb_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 92.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 92.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 92.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.824999988079071}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   bias=False)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatv2nb_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  237 K │
└───┴────────┴────────────┴────────┘

Trainable params: 237 K                                                                      
Non-trainable params: 0                                                                      
Total params: 237 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7070000171661377}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 attention_type=oxgat.components.GATv2AttentionHead,
                                                 bias=False)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatv2nb_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 32.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 32.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 32.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7799999713897705}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   bias=False)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatv2nb_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  562 K │
└───┴────────┴────────────┴────────┘

Trainable params: 562 K                                                                      
Non-trainable params: 0                                                                      
Total params: 562 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.27399998903274536}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                attention_type=oxgat.components.GATv2AttentionHead,
                                                bias=False)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatv2nb_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 48.3 K │
└───┴────────┴────────────┴────────┘

Trainable params: 48.3 K                                                                     
Non-trainable params: 0                                                                      
Total params: 48.3 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8619999885559082}

--------------------------------------------------------------------------------

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    attention_type=oxgat.components.GATv2AttentionHead,
                                                    bias=False)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatv2nb_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 49.9 K │
└───┴────────┴────────────┴────────┘

Trainable params: 49.9 K                                                                     
Non-trainable params: 0                                                                      
Total params: 49.9 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.734000027179718}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  bias=False)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatv2nb_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  436 K │
└───┴────────┴────────────┴────────┘

Trainable params: 436 K                                                                      
Non-trainable params: 0                                                                      
Total params: 436 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9139999747276306}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  bias=False)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatv2nb_results["physics"] = physics_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  539 K │
└───┴────────┴────────────┴────────┘

Trainable params: 539 K                                                                      
Non-trainable params: 0                                                                      
Total params: 539 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9179999828338623}

--------------------------------------------------------------------------------

In [5]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  bias=False)
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatv2nb_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  1.2 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 1.9 K                                                                      
Non-trainable params: 0                                                                      
Total params: 1.9 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6791961992313487}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatv2nb_results.json", "w") as f:
    json.dump(gatv2nb_results, f)

## GATv2 (source weights, bias)

In [ ]:
gatv2source_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               attention_type=oxgat.components.GATv2AttentionHead,
                                               weight_sharing=False)
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatv2source_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  184 K │
└───┴────────┴────────────┴────────┘

Trainable params: 184 K                                                                      
Non-trainable params: 0                                                                      
Total params: 184 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8289999961853027}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   weight_sharing=False)
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatv2source_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  475 K │
└───┴────────┴────────────┴────────┘

Trainable params: 475 K                                                                      
Non-trainable params: 0                                                                      
Total params: 475 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7099999785423279}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 attention_type=oxgat.components.GATv2AttentionHead,
                                                 weight_sharing=False)
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatv2source_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 64.7 K │
└───┴────────┴────────────┴────────┘

Trainable params: 64.7 K                                                                     
Non-trainable params: 0                                                                      
Total params: 64.7 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.777999997138977}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   weight_sharing=False)
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatv2source_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  1.1 M │
└───┴────────┴────────────┴────────┘

Trainable params: 1.1 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.1 M                                                                          
Total estimated model params size (MB): 4

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.30799999833106995}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                attention_type=oxgat.components.GATv2AttentionHead,
                                                weight_sharing=False)
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatv2source_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 96.7 K │
└───┴────────┴────────────┴────────┘

Trainable params: 96.7 K                                                                     
Non-trainable params: 0                                                                      
Total params: 96.7 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8659999966621399}

--------------------------------------------------------------------------------

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    attention_type=oxgat.components.GATv2AttentionHead,
                                                    weight_sharing=False)
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatv2source_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 99.8 K │
└───┴────────┴────────────┴────────┘

Trainable params: 99.8 K                                                                     
Non-trainable params: 0                                                                      
Total params: 99.8 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.593999981880188}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False)
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatv2source_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  873 K │
└───┴────────┴────────────┴────────┘

Trainable params: 873 K                                                                      
Non-trainable params: 0                                                                      
Total params: 873 K                                                                          
Total estimated model params size (MB): 3

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9079999923706055}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False)
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatv2source_results["physics"] = physics_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  1.1 M │
└───┴────────┴────────────┴────────┘

Trainable params: 1.1 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.1 M                                                                          
Total estimated model params size (MB): 4

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.921999990940094}

--------------------------------------------------------------------------------

In [9]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False)
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatv2source_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  2.5 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 3.2 K                                                                      
Non-trainable params: 0                                                                      
Total params: 3.2 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6952104134880936}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatv2source_results.json", "w") as f:
    json.dump(gatv2source_results, f)

## GATv2 (all separate weights, bias)

In [ ]:
gatv2sep_results = {}

In [ ]:
# Cora
cora_model = oxgat.models.CustomNodeClassifier(in_features=1433,
                                               num_classes=7,
                                               attention_type=oxgat.components.GATv2AttentionHead,
                                               weight_sharing=False,
                                               feature_update_matrix="separate")
cora_model.standard_train(cora, use_gpu=True)
cora_model.standard_test(cora)
gatv2sep_results["cora"] = cora_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  276 K │
└───┴────────┴────────────┴────────┘

Trainable params: 276 K                                                                      
Non-trainable params: 0                                                                      
Total params: 276 K                                                                          
Total estimated model params size (MB): 1

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8399999737739563}

--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [ ]:
# CiteSeer
citeseer_model = oxgat.models.CustomNodeClassifier(in_features=3703,
                                                   num_classes=6,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   weight_sharing=False,
                                                   feature_update_matrix="separate")
citeseer_model.standard_train(citeseer, use_gpu=True)
citeseer_model.standard_test(citeseer)
gatv2sep_results["citeseer"] = citeseer_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  712 K │
└───┴────────┴────────────┴────────┘

Trainable params: 712 K                                                                      
Non-trainable params: 0                                                                      
Total params: 712 K                                                                          
Total estimated model params size (MB): 2

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7080000042915344}

--------------------------------------------------------------------------------

In [ ]:
# PubMed
pubmed_model = oxgat.models.CustomNodeClassifier(in_features=500,
                                                 num_classes=3,
                                                 attention_type=oxgat.components.GATv2AttentionHead,
                                                 weight_sharing=False,
                                                 feature_update_matrix="separate")
pubmed_model.standard_train(pubmed, use_gpu=True)
pubmed_model.standard_test(pubmed)
gatv2sep_results["pubmed"] = pubmed_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │ 96.9 K │
└───┴────────┴────────────┴────────┘

Trainable params: 96.9 K                                                                     
Non-trainable params: 0                                                                      
Total params: 96.9 K                                                                         
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7839999794960022}

--------------------------------------------------------------------------------

In [ ]:
# CoraFull
corafull_model = oxgat.models.CustomNodeClassifier(in_features=8710,
                                                   num_classes=70,
                                                   attention_type=oxgat.components.GATv2AttentionHead,
                                                   weight_sharing=False,
                                                   feature_update_matrix="separate")
corafull_model.standard_train(corafull, use_gpu=True)
corafull_model.standard_test(corafull)
gatv2sep_results["corafull"] = corafull_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  1.7 M │
└───┴────────┴────────────┴────────┘

Trainable params: 1.7 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.7 M                                                                          
Total estimated model params size (MB): 6

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.28600001335144043}

--------------------------------------------------------------------------------

In [ ]:
# Photo
photo_model = oxgat.models.CustomNodeClassifier(in_features=745,
                                                num_classes=8,
                                                attention_type=oxgat.components.GATv2AttentionHead,
                                                weight_sharing=False,
                                                feature_update_matrix="separate")
photo_model.standard_train(photo, use_gpu=True)
photo_model.standard_test(photo)
gatv2sep_results["photo"] = photo_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  144 K │
└───┴────────┴────────────┴────────┘

Trainable params: 144 K                                                                      
Non-trainable params: 0                                                                      
Total params: 144 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.8579999804496765}

--------------------------------------------------------------------------------

In [ ]:
# Computers
computers_model = oxgat.models.CustomNodeClassifier(in_features=767,
                                                    num_classes=10,
                                                    attention_type=oxgat.components.GATv2AttentionHead,
                                                    weight_sharing=False,
                                                    feature_update_matrix="separate")
computers_model.standard_train(computers, use_gpu=True)
computers_model.standard_test(computers)
gatv2sep_results["computers"] = computers_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  149 K │
└───┴────────┴────────────┴────────┘

Trainable params: 149 K                                                                      
Non-trainable params: 0                                                                      
Total params: 149 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.7419999837875366}

--------------------------------------------------------------------------------

In [ ]:
# Compsci
compsci_model = oxgat.models.CustomNodeClassifier(in_features=6805,
                                                  num_classes=15,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False,
                                                  feature_update_matrix="separate")
compsci_model.standard_train(compsci, use_gpu=True)
compsci_model.standard_test(compsci)
gatv2sep_results["compsci"] = compsci_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name   ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers │ ModuleList │  1.3 M │
└───┴────────┴────────────┴────────┘

Trainable params: 1.3 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.3 M                                                                          
Total estimated model params size (MB): 5

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.9039999842643738}

--------------------------------------------------------------------------------

In [ ]:
# Physics
physics_model = oxgat.models.CustomNodeClassifier(in_features=8415,
                                                  num_classes=5,
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False,
                                                  feature_update_matrix="separate")
physics_model.standard_train(physics, use_gpu=True)
physics_model.standard_test(physics)
gatv2sep_results["physics"] = physics_model.final_metrics

In [10]:
# Molhiv
molhiv_model = oxgat.models.CustomGraphClassifier(in_features=9,
                                                  num_classes=2,
                                                  cast_to_float=True,
                                                  mlp_hidden_layer_dims=64,
                                                  dropout=0,
                                                  metric="roc_auc_score",
                                                  restore_best="acc",
                                                  attention_type=oxgat.components.GATv2AttentionHead,
                                                  weight_sharing=False,
                                                  feature_update_matrix="separate")
molhiv_model.standard_train(molhiv_train, molhiv_val, use_gpu=True)
molhiv_model.standard_test(molhiv_test)
gatv2sep_results["molhiv"] = molhiv_model.final_metrics

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ attention_layers │ ModuleList │  3.6 K │
│ 1 │ mlp              │ Sequential │    706 │
└───┴──────────────────┴────────────┴────────┘

Trainable params: 4.3 K                                                                      
Non-trainable params: 0                                                                      
Total params: 4.3 K                                                                          
Total estimated model params size (MB): 0

Output()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: 
UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the 
number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

{'test_acc': 0.6087313389598099}

--------------------------------------------------------------------------------

In [ ]:
with open("results/gatv2sep_results.json", "w") as f:
    json.dump(gatv2sep_results, f)